In [1]:
import requests  # Importa la biblioteca para realizar solicitudes HTTP
import json  # Importa la biblioteca para manejar datos en formato JSON

In [2]:

def request_tickets(issues, cliente_busqueda, prioridad_busqueda, status_busqueda, page, sort_order):
    """
    Función para obtener tickets de Jira mediante su API REST.
    
    Parámetros:
    - issues: Lista donde se almacenarán los tickets obtenidos
    - cliente_busqueda: Filtro por cliente
    - prioridad_busqueda: Filtro por prioridad
    - status_busqueda: Filtro por estado del ticket
    - page: Número de página para la paginación
    - sort_order: Criterio de ordenamiento
    """
    print(f"Buscando tickets para el cliente: {cliente_busqueda}")
    # Configuración del cliente REST con la URL base de Jira
    base_url = "https://bajaware.atlassian.net"
    # Credenciales de autenticación para la API de Jira (usuario y token)
    auth = ("dmoncivais@bajaware.com", "ATATT3xFfGF0j2KBKb7Pqq1T4oJtuX84Zb9YpfLsc4c6ZRBkgOiljgIswRATjWMksHPbZLXy5aJzMya1YKJKisfLYOoyW4Rb6fq1PAi7__JnLGcRr7zQeG1xRUt6NXOF6WK7eora0pX6TlFRIS3WH66baE8ja665Mn9hGpzNpn19IHZuuvCKfPU=59FC5052")
    print(f"Autenticación exitosa")
    # Inicializa el contador total de tickets
    total = 0
    # Comienza a construir la URL de la consulta JQL para el proyecto "BS"
    req = "/rest/api/3/search?jql=project=\"BS\""
    print(f"URL de la consulta: {base_url}{req}")
    # Agrega parámetros de ordenamiento si se especificaron
    if sort_order != "":
        req += "+order+By+"  # Añade la cláusula de ordenamiento
        sort = sort_order.split('_')  # Divide el criterio de ordenamiento
        req += sort[0] + "+"  # Agrega el campo por el cual ordenar
        if len(sort) > 1:  # Si hay un segundo elemento, indica orden descendente
            req += "DESC"
    print(f"Criterio de ordenamiento: {sort_order}")
    # Agrega filtros a la consulta si se especificaron
    if cliente_busqueda != "":
        # Filtro por cliente usando el campo personalizado 10500
        req += f"&\"cf[10500]\"=\"{cliente_busqueda}\""
    if prioridad_busqueda != "":
        # Filtro por prioridad usando el campo personalizado 11208
        req += f"&\"cf[11208]\"=\"{prioridad_busqueda}\""
    if status_busqueda != "":
        # Filtro por estado del ticket
        req += f"&\"status\"=\"{status_busqueda}\""
    print(f"Filtros aplicados: {req}")
    # Configura la paginación
    start = page - 1 if page == 1 else (page - 1) * 100  # Calcula el índice de inicio
    # Agrega parámetros adicionales: fecha de creación, campos a devolver, resultados máximos e índice de inicio
    req += f"&\"created\">=\"2018/12/31\"&fields=customfield_10500,key,status,created,summary,resolved,resolution,assignee,customfield_11208,customfield_10705&maxResults={-1}&startAt={start}"
    print(f"URL de la consulta con filtros: {base_url}{req}")
    # Ejecuta la solicitud HTTP GET a la API de Jira
    response = requests.get(f"{base_url}{req}", auth=auth)
    print(f"Respuesta de la consulta: {response.status_code}")
    # Procesa la respuesta
    if response.status_code == 200:  # Si la solicitud fue exitosa
        data = response.json()  # Convierte la respuesta JSON a un diccionario Python
        total = data["total"]  # Obtiene el número total de tickets
        issues.extend(data["issues"])  # Agrega los tickets a la lista proporcionada
    else:
        # Si la solicitud falló, imprime información de error
        print(f"Request failed: {response.status_code}")
        print(response.text)
        return -1  # Retorna -1 para indicar error
    
    # Retorna el número total de tickets encontrados
    return total

In [3]:
issues = []
cliente_busqueda = ""
prioridad_busqueda = ""
status_busqueda = ""
page = 1
sort_order = ""

total_issues = request_tickets(issues, cliente_busqueda, prioridad_busqueda, status_busqueda, page, sort_order)

print(f"Total de tickets encontrados: {total_issues}")
print(f"Tickets obtenidos: {len(issues)}")

# Guardar los tickets en un archivo JSON
with open('tickets.json', 'w', encoding='utf-8') as f:
    json.dump(issues, f, ensure_ascii=False, indent=4)

print("✅ Tickets guardados en tickets.json")

Buscando tickets para el cliente: 
Autenticación exitosa
URL de la consulta: https://bajaware.atlassian.net/rest/api/3/search?jql=project="BS"
Criterio de ordenamiento: 
Filtros aplicados: /rest/api/3/search?jql=project="BS"
URL de la consulta con filtros: https://bajaware.atlassian.net/rest/api/3/search?jql=project="BS"&"created">="2018/12/31"&fields=customfield_10500,key,status,created,summary,resolved,resolution,assignee,customfield_11208,customfield_10705&maxResults=-1&startAt=0
Respuesta de la consulta: 200
Total de tickets encontrados: 5900
Tickets obtenidos: 5900
✅ Tickets guardados en tickets.json
